In [ ]:
!pip install spark
!pip install pyspark
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 34 kB/s 
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58761 sha256=41d3f287b20803c5fd6f2e7d49ac1d64647b7e713c0c7c52c254b25f8dc20ad1
  Stored in directory: /root/.cache/pip/wheels/c5/19/ff/9b16f354528bc9698ec3286be7947ebbf1f8391325553961d4
Successfully built spark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 46 kB/s 
     |████████████████████████████████| 199 kB 46.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=f92796921510be69dd3a0454d74bf6b5eb44cc9f3523cc83c21c194e2898f086
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-p

In [ ]:
import findspark
findspark.init()
import pyspark as ps
import warnings
from pyspark.sql import SQLContext

In [ ]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import Word2Vec
import pandas as pd

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


##Приведение данных к стандартному шаблону

In [ ]:
#change file path
pd_train = pd.read_csv('gdrive/My Drive/MADE/HPC_BD/train.csv')
pd_train.head(15)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [ ]:
pd_train['comment_text'].replace(r'\s+|\\n', ' ', regex=True, inplace=True)
pd_train.head(15)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation Why the edits made under my userna...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,""" More I can't make any real suggestions on im...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,""" Congratulations from me as well, use the too...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [ ]:
pd_train.to_csv('gdrive/My Drive/MADE/HPC_BD/pd_train.csv')

##Подключения к SparkContext

In [ ]:
sc = None
sqlContext = None

try:
    # create SparkContext on all CPUs available: in my case I have 4 CPUs on my laptop
    sc = ps.SparkContext('local[4]')
    sqlContext = SQLContext(sc)
    print("Just created a SparkContext")
except ValueError:
    warnings.warn("SparkContext already exists in this scope")

/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Just created a SparkContext


In [ ]:
#change file path
df_train = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true',mode ="DROPMALFORMED").load('gdrive/My Drive/MADE/HPC_BD/pd_train.csv')
df_train.show(15)

+---+----------------+--------------------+-----+------------+-------+------+------+-------------+
|_c0|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|
+---+----------------+--------------------+-----+------------+-------+------+------+-------------+
|  0|0000997932d777bf|Explanation Why t...|    0|           0|      0|     0|     0|            0|
|  1|000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|
|  2|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|
|  4|0001d958c54c6e35|You, sir, are my ...|    0|           0|      0|     0|     0|            0|
|  6|0002bcb3da6cb337|COCKSUCKER BEFORE...|    1|           1|      1|     0|     1|            0|
|  7|00031b1e95af7921|Your vandalism to...|    0|           0|      0|     0|     0|            0|
|  8|00037261f536c51d|Sorry if the word...|    0|           0|      0|     0|     0|            0|
|  9|00040

In [ ]:
#разделение данных
(train_set, val_set, test_set) = df_train.randomSplit([0.98, 0.01, 0.01], seed = 2000)

## HashingTF и IDF

###toxic

In [ ]:
#toxic
tokenizer_toxic = Tokenizer(inputCol="comment_text", outputCol="words")
hashtf_toxic = HashingTF(numFeatures=2**16, inputCol="words", outputCol='tf')
idf_toxic = IDF(inputCol='tf', outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
label_stringIdx_toxic = StringIndexer(inputCol = "toxic", outputCol = "label")
pipeline_toxic = Pipeline(stages=[tokenizer_toxic, hashtf_toxic, idf_toxic, label_stringIdx_toxic])

pipelineFit_toxic = pipeline_toxic.fit(train_set)
train_df_toxic = pipelineFit_toxic.transform(train_set)
val_df_toxic = pipelineFit_toxic.transform(val_set)
test_df_toxic = pipelineFit_toxic.transform(test_set)
train_df_toxic.show(5)

+---+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+-----+
|_c0|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|               words|                  tf|            features|label|
+---+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+-----+
|  0|0000997932d777bf|Explanation Why t...|    0|           0|      0|     0|     0|            0|[explanation, why...|(65536,[763,1880,...|(65536,[763,1880,...|  0.0|
|  1|000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|[d'aww!, he, matc...|(65536,[2195,4714...|(65536,[2195,4714...|  0.0|
|  2|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|[hey, man,, i'm, ...|(65536,[6589,1001...|(65536,[6589,1001...

In [ ]:
lr_toxic = LogisticRegression(maxIter=100)
lrModel_toxic = lr_toxic.fit(train_df_toxic)
predictions_toxic = lrModel_toxic.transform(val_df_toxic)
test_toxic = lrModel_toxic.transform(test_df_toxic)

In [ ]:
predictions_toxic

DataFrame[_c0: int, id: string, comment_text: string, toxic: string, severe_toxic: string, obscene: string, threat: string, insult: string, identity_hate: string, words: array<string>, tf: vector, features: vector, label: double, rawPrediction: vector, probability: vector, prediction: double]

In [ ]:
test_toxic

DataFrame[_c0: int, id: string, comment_text: string, toxic: string, severe_toxic: string, obscene: string, threat: string, insult: string, identity_hate: string, words: array<string>, tf: vector, features: vector, label: double, rawPrediction: vector, probability: vector, prediction: double]

In [ ]:
#PySpark встроенная метрика areaUnderROC
evaluator_toxic = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator_toxic.evaluate(predictions_toxic)

0.8326975892584698

In [ ]:
#PySpark встроенная метрика areaUnderROC
evaluator_toxic_test = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator_toxic_test.evaluate(test_toxic)

0.8168055400277524

In [ ]:
evaluator_toxic.getMetricName()

'areaUnderROC'

In [ ]:
predictions_toxic.show(5)

+---+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|_c0|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|               words|                  tf|            features|label|       rawPrediction|         probability|prediction|
+---+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|182|006eaaaca322e12d|""") (ETA: John D...|    0|           0|      0|     0|     0|            0|["""), (eta:, joh...|(65536,[19271,267...|(65536,[19271,267...|  0.0|[-11.311266554283...|[1.22341567017678...|       1.0|
|306|00c41d6b52961339|Will you make the...|    0|           0|      0|     0|     0|            0|[will, you, make,.

In [ ]:
#accuracy_toxic_val
accuracy_toxic = predictions_toxic.filter(predictions_toxic.label == predictions_toxic.prediction).count() / float(val_set.count())
accuracy_toxic

0.8599827139152982

In [ ]:
#accuracy_toxic_test
accuracy_toxic_test = test_toxic.filter(test_toxic.label == test_toxic.prediction).count() / float(test_set.count())
accuracy_toxic_test

0.8488773747841105

**accuracy на валидации 0.86, accuracy на тесте 0.85. Результаты на тесте и влидации не силно отличаются**

###severe_toxic

In [ ]:
#severe_toxic
tokenizer_severe_toxic = Tokenizer(inputCol="comment_text", outputCol="words")
hashtf_severe_toxic = HashingTF(numFeatures=2**16, inputCol="words", outputCol='tf')
idf_severe_toxic = IDF(inputCol='tf', outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
label_stringIdx_severe_toxic = StringIndexer(inputCol = "severe_toxic", outputCol = "label")
pipeline_severe_toxic = Pipeline(stages=[tokenizer_severe_toxic, hashtf_severe_toxic, idf_severe_toxic, label_stringIdx_severe_toxic])

pipelineFit_severe_toxic = pipeline_severe_toxic.fit(train_set)
train_df_severe_toxic = pipelineFit_severe_toxic.transform(train_set)
val_df_severe_toxic = pipelineFit_severe_toxic.transform(val_set)
test_df_severe_toxic = pipelineFit_severe_toxic.transform(test_set)
train_df_severe_toxic.show(5)

+---+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+-----+
|_c0|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|               words|                  tf|            features|label|
+---+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+-----+
|  0|0000997932d777bf|Explanation Why t...|    0|           0|      0|     0|     0|            0|[explanation, why...|(65536,[763,1880,...|(65536,[763,1880,...|  0.0|
|  1|000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|[d'aww!, he, matc...|(65536,[2195,4714...|(65536,[2195,4714...|  0.0|
|  2|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|[hey, man,, i'm, ...|(65536,[6589,1001...|(65536,[6589,1001...

In [ ]:
lr_severe_toxic = LogisticRegression(maxIter=100)
lrModel_severe_toxic = lr_severe_toxic.fit(train_df_severe_toxic)
predictions_severe_toxic = lrModel_severe_toxic.transform(val_df_severe_toxic)
test_severe_toxic = lrModel_severe_toxic.transform(test_df_severe_toxic)

In [ ]:
evaluator_severe_toxic = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator_severe_toxic.evaluate(predictions_severe_toxic)

0.7425363098440031

In [ ]:
evaluator_severe_toxic_test = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator_severe_toxic_test.evaluate(test_severe_toxic)

0.6670862521891413

In [ ]:
evaluator_severe_toxic_test.getMetricName()

'areaUnderROC'

In [ ]:
#accuracy_severe_toxic_val
accuracy_severe_toxic = predictions_severe_toxic.filter(predictions_severe_toxic.label == predictions_severe_toxic.prediction).count() / float(val_set.count())
accuracy_severe_toxic

0.9446845289541919

In [ ]:
#accuracy_toxic_test
accuracy_toxic_test = test_toxic.filter(test_toxic.label == test_toxic.prediction).count() / float(test_set.count())
accuracy_toxic_test

0.8488773747841105

**accuracy на валидации 0.94, accuracy на тесте 0.85. Результаты на тесте и влидации не силно отличаются, модель не переобучилась**

###obscene

In [ ]:
#obscene

tokenizer_obscene = Tokenizer(inputCol="comment_text", outputCol="words")
hashtf_obscene = HashingTF(numFeatures=2**16, inputCol="words", outputCol='tf')
idf_obscene = IDF(inputCol='tf', outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
label_stringIdx_obscene = StringIndexer(inputCol = "obscene", outputCol = "label")
pipeline_obscene = Pipeline(stages=[tokenizer_obscene, hashtf_obscene, idf_obscene, label_stringIdx_obscene])

pipelineFit_obscene = pipeline_obscene.fit(train_set)
train_df_obscene = pipelineFit_obscene.transform(train_set)
val_df_obscene = pipelineFit_obscene.transform(val_set)
test_df_obscene = pipelineFit_obscene.transform(test_set)
train_df_obscene.show(5)

+---+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+-----+
|_c0|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|               words|                  tf|            features|label|
+---+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+-----+
|  0|0000997932d777bf|Explanation Why t...|    0|           0|      0|     0|     0|            0|[explanation, why...|(65536,[763,1880,...|(65536,[763,1880,...|  0.0|
|  1|000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|[d'aww!, he, matc...|(65536,[2195,4714...|(65536,[2195,4714...|  0.0|
|  2|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|[hey, man,, i'm, ...|(65536,[6589,1001...|(65536,[6589,1001...

In [ ]:
lr_obscene = LogisticRegression(maxIter=100)
lrModel_obscene = lr_obscene.fit(train_df_obscene)
predictions_obscene = lrModel_obscene.transform(val_df_obscene)
test_obscene = lrModel_obscene.transform(test_df_obscene)

In [ ]:
evaluator_obscene.getMetricName()

'areaUnderROC'

In [ ]:
evaluator_obscene = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator_obscene.evaluate(predictions_obscene)

0.7972652689152232

In [ ]:
evaluator_obscene_test = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator_obscene_test.evaluate(test_obscene)

0.7647273468774491

In [ ]:
#accuracy_obscene_val
accuracy_obscene = predictions_obscene.filter(predictions_obscene.label == predictions_obscene.prediction).count() / float(val_set.count())
accuracy_obscene

0.8980121002592912

In [ ]:
#accuracy_obscene_test
accuracy_obscene_test = test_obscene.filter(test_obscene.label == test_obscene.prediction).count() / float(test_set.count())
accuracy_obscene_test

0.8773747841105354

**accuracy на валидации 0.9, accuracy на тесте 0.88. Результаты на тесте и влидации не силно отличаются, модель не переобучилась**

###threat

In [ ]:
#threat

tokenizer_threat = Tokenizer(inputCol="comment_text", outputCol="words")
hashtf_threat = HashingTF(numFeatures=2**16, inputCol="words", outputCol='tf')
idf_threat = IDF(inputCol='tf', outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
label_stringIdx_threat = StringIndexer(inputCol = "threat", outputCol = "label")
pipeline_threat = Pipeline(stages=[tokenizer_threat, hashtf_threat, idf_threat, label_stringIdx_threat])

pipelineFit_threat = pipeline_threat.fit(train_set)
train_df_threat = pipelineFit_threat.transform(train_set)
val_df_threat = pipelineFit_threat.transform(val_set)
test_df_threat = pipelineFit_threat.transform(test_set)
train_df_threat.show(5)

+---+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+-----+
|_c0|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|               words|                  tf|            features|label|
+---+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+-----+
|  0|0000997932d777bf|Explanation Why t...|    0|           0|      0|     0|     0|            0|[explanation, why...|(65536,[763,1880,...|(65536,[763,1880,...|  0.0|
|  1|000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|[d'aww!, he, matc...|(65536,[2195,4714...|(65536,[2195,4714...|  0.0|
|  2|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|[hey, man,, i'm, ...|(65536,[6589,1001...|(65536,[6589,1001...

In [ ]:
lr_threat = LogisticRegression(maxIter=100)
lrModel_threat = lr_threat.fit(train_df_threat)
predictions_threat = lrModel_threat.transform(val_df_threat)
test_threat = lrModel_threat.transform(test_df_threat)

In [ ]:
evaluator_threat = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator_threat.evaluate(predictions_threat)

0.9831168831168832

In [ ]:
#accuracy_threat_val
accuracy_threat = predictions_threat.filter(predictions_threat.label == predictions_threat.prediction).count() / float(val_set.count())
accuracy_threat

0.9714779602420052

In [ ]:
#accuracy_threat_test
accuracy_threat_test = test_threat.filter(test_threat.label == test_threat.prediction).count() / float(test_set.count())
accuracy_threat_test

0.9706390328151986

**accuracy на валидации 0.97, accuracy на тесте 0.97. Результаты на тесте и влидации не силно отличаются, модель не переобучилась**

###insult

In [ ]:
#insult

tokenizer_insult = Tokenizer(inputCol="comment_text", outputCol="words")
hashtf_insult = HashingTF(numFeatures=2**16, inputCol="words", outputCol='tf')
idf_insult = IDF(inputCol='tf', outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
label_stringIdx_insult = StringIndexer(inputCol = "insult", outputCol = "label")
pipeline_insult = Pipeline(stages=[tokenizer_insult, hashtf_insult, idf_insult, label_stringIdx_insult])

pipelineFit_insult = pipeline_insult.fit(train_set)
train_df_insult = pipelineFit_insult.transform(train_set)
val_df_insult = pipelineFit_insult.transform(val_set)
test_df_insult = pipelineFit_insult.transform(test_set)
train_df_insult.show(5)

+---+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+-----+
|_c0|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|               words|                  tf|            features|label|
+---+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+-----+
|  0|0000997932d777bf|Explanation Why t...|    0|           0|      0|     0|     0|            0|[explanation, why...|(65536,[763,1880,...|(65536,[763,1880,...|  0.0|
|  1|000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|[d'aww!, he, matc...|(65536,[2195,4714...|(65536,[2195,4714...|  0.0|
|  2|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|[hey, man,, i'm, ...|(65536,[6589,1001...|(65536,[6589,1001...

In [ ]:
lr_insult = LogisticRegression(maxIter=100)
lrModel_insult = lr_insult.fit(train_df_insult)
predictions_insult = lrModel.transform(val_df_insult)
test_insult = lrModel_insult.transform(test_df_insult)

In [ ]:
evaluator_insult = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator_insult.evaluate(predictions_insult)

0.7726278370248826

In [ ]:
evaluator_insult_test = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator_insult_test.evaluate(test_insult)

0.787600933942991

In [ ]:
#accuracy_insult_val
accuracy_insult = predictions_insult.filter(predictions_insult.label == predictions_insult.prediction).count() / float(val_set.count())
accuracy_insult

0.8928262748487468

In [ ]:
#accuracy_insult_test
accuracy_insult_test = test_insult.filter(test_insult.label == test_insult.prediction).count() / float(test_set.count())
accuracy_insult_test

0.8756476683937824

**accuracy на валидации 0.89, accuracy на тесте 0.88. Результаты на тесте и влидации не силно отличаются, модель не переобучилась**

###identity_hate

In [ ]:
#identity_hate

tokenizer_identity_hate = Tokenizer(inputCol="comment_text", outputCol="words")
hashtf_identity_hate = HashingTF(numFeatures=2**16, inputCol="words", outputCol='tf')
idf_identity_hate = IDF(inputCol='tf', outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
label_stringIdx_identity_hate = StringIndexer(inputCol = "identity_hate", outputCol = "label")
pipeline_identity_hate = Pipeline(stages=[tokenizer_identity_hate, hashtf_identity_hate, idf_identity_hate, label_stringIdx_identity_hate])

pipelineFit_identity_hate = pipeline_identity_hate.fit(train_set)
train_df_identity_hate = pipelineFit_identity_hate.transform(train_set)
val_df_identity_hate = pipelineFit_identity_hate.transform(val_set)
test_df_identity_hate = pipelineFit_identity_hate.transform(test_set)
train_df_identity_hate.show(5)

+---+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+-----+
|_c0|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|               words|                  tf|            features|label|
+---+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+-----+
|  0|0000997932d777bf|Explanation Why t...|    0|           0|      0|     0|     0|            0|[explanation, why...|(65536,[763,1880,...|(65536,[763,1880,...|  0.0|
|  1|000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|[d'aww!, he, matc...|(65536,[2195,4714...|(65536,[2195,4714...|  0.0|
|  2|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|[hey, man,, i'm, ...|(65536,[6589,1001...|(65536,[6589,1001...

In [ ]:
lr_identity_hate = LogisticRegression(maxIter=100)
lrModel_identity_hate = lr_identity_hate.fit(train_df_identity_hate)
predictions_identity_hate = lrModel_identity_hate.transform(val_df_identity_hate)
test_identity_hate = lrModel_identity_hate.transform(test_df_identity_hate)

In [ ]:
evaluator_identity_hate = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator_identity_hate.evaluate(predictions_identity_hate)

0.6393894566971485

In [ ]:
evaluator_identity_hate_test = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator_identity_hate_test.evaluate(test_identity_hate)

0.6155591070784134

In [ ]:
#accuracy_identity_val
accuracy_identity_hate = predictions_identity_hate.filter(predictions_identity_hate.label == predictions_identity_hate.prediction).count() / float(val_set.count())
accuracy_identity_hate

0.9386343993085566

In [ ]:
#accuracy_identity_test
accuracy_identity_hate_test = test_identity_hate.filter(test_identity_hate.label == test_identity_hate.prediction).count() / float(test_set.count())
accuracy_identity_hate_test

0.9525043177892919

**accuracy на валидации 0.94, accuracy на тесте 0.95. Результаты на тесте и влидации не силно отличаются, модель не переобучилась**

#Word2Vec

In [ ]:
tokenizer = Tokenizer(inputCol="comment_text", outputCol="words")
word2Vec = Word2Vec(vectorSize=10, minCount=0, inputCol="words", outputCol="result_w2v")
pipeline = Pipeline(stages=[tokenizer, word2Vec])
pipelineFit = pipeline.fit(train_set)

train_df_w2v = pipelineFit.transform(train_set)
val_df_w2v = pipelineFit.transform(val_set)
test_df_w2v = pipelineFit.transform(test_set)
test_df_w2v.show(5)

+---+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+
|_c0|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|               words|          result_w2v|
+---+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+
|136|00562e78e0b34102|Yeah, let's merge...|    0|           0|      0|     0|     0|            0|[yeah,, let's, me...|[-0.1298364839458...|
|138|0057e30091cf3e81|""" A cookie for ...|    0|           0|      0|     0|     0|            0|[""", a, cookie, ...|[0.01772472597658...|
|298|00c0b8ed05ed7833|Fuck off You are ...|    1|           0|      1|     0|     0|            0|[fuck, off, you, ...|[-0.2699484359990...|
|342|00db176890cd71bf|Thank you Hei, it...|    1|           0|      0|     0|     0|            0|[thank, you, hei,...|[-0.1332822859434...|
|602|019ab9a9

In [ ]:
from pyspark.sql.types import *

# Change column type
train_df_w2v = train_df_w2v.withColumn("toxic", train_df_w2v["toxic"].cast(IntegerType()))
train_df_w2v = train_df_w2v.withColumn("severe_toxic", train_df_w2v["severe_toxic"].cast(IntegerType()))
train_df_w2v = train_df_w2v.withColumn("obscene", train_df_w2v["obscene"].cast(IntegerType()))
train_df_w2v = train_df_w2v.withColumn("threat", train_df_w2v["threat"].cast(IntegerType()))
train_df_w2v = train_df_w2v.withColumn("insult", train_df_w2v["insult"].cast(IntegerType()))
train_df_w2v = train_df_w2v.withColumn("identity_hate", train_df_w2v["identity_hate"].cast(IntegerType()))

test_df_w2v = test_df_w2v.withColumn("toxic", test_df_w2v["toxic"].cast(IntegerType()))
test_df_w2v = test_df_w2v.withColumn("severe_toxic", test_df_w2v["severe_toxic"].cast(IntegerType()))
test_df_w2v = test_df_w2v.withColumn("obscene", test_df_w2v["obscene"].cast(IntegerType()))
test_df_w2v = test_df_w2v.withColumn("threat", test_df_w2v["threat"].cast(IntegerType()))
test_df_w2v = test_df_w2v.withColumn("insult", test_df_w2v["insult"].cast(IntegerType()))
test_df_w2v = test_df_w2v.withColumn("identity_hate", test_df_w2v["identity_hate"].cast(IntegerType()))

val_df_w2v = val_df_w2v.withColumn("toxic", val_df_w2v["toxic"].cast(IntegerType()))
val_df_w2v = val_df_w2v.withColumn("severe_toxic", val_df_w2v["severe_toxic"].cast(IntegerType()))
val_df_w2v = val_df_w2v.withColumn("obscene", val_df_w2v["obscene"].cast(IntegerType()))
val_df_w2v = val_df_w2v.withColumn("threat", val_df_w2v["threat"].cast(IntegerType()))
val_df_w2v = val_df_w2v.withColumn("insult", val_df_w2v["insult"].cast(IntegerType()))
val_df_w2v = val_df_w2v.withColumn("identity_hate", val_df_w2v["identity_hate"].cast(IntegerType()))

In [ ]:
train_df_w2v.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- id: string (nullable = true)
 |-- comment_text: string (nullable = true)
 |-- toxic: integer (nullable = true)
 |-- severe_toxic: integer (nullable = true)
 |-- obscene: integer (nullable = true)
 |-- threat: integer (nullable = true)
 |-- insult: integer (nullable = true)
 |-- identity_hate: integer (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- result_w2v: vector (nullable = true)



###toxic

In [ ]:
mlor_toxic = LogisticRegression(featuresCol='result_w2v', labelCol='toxic')
mlorModel_toxic = mlor_toxic.fit(train_df_w2v)

In [ ]:
predictions_toxic = mlorModel_toxic.transform(val_df_w2v)

In [ ]:
test_toxic = mlorModel_toxic.transform(test_df_w2v)

In [ ]:
#accuracy_val
accuracy_toxic = predictions_toxic.filter(predictions_toxic.toxic == predictions_toxic.prediction).count() / float(val_set.count())
accuracy_toxic

0.9291270527225584

In [ ]:
#accuracy_test
accuracy_toxic_test = test_toxic.filter(test_toxic.toxic == test_toxic.prediction).count() / float(test_set.count())
accuracy_toxic_test

0.9205526770293609

**accuracy на валидации 0.93, accuracy на тесте 0.92. Результаты на тесте и влидации не силно отличаются, модель не переобучилась**

###severe_toxic

In [ ]:
mlor_severe_toxic = LogisticRegression(featuresCol='result_w2v', labelCol='toxic')
mlorModel_severe_toxic = mlor_severe_toxic.fit(train_df_w2v)

In [ ]:
predictions_severe_toxic = mlorModel_severe_toxic.transform(val_df_w2v)

In [ ]:
test_severe_toxic = mlorModel_severe_toxic.transform(test_df_w2v)

In [ ]:
#accuracy_val
accuracy_severe_toxic = predictions_severe_toxic.filter(predictions_severe_toxic.severe_toxic == predictions_severe_toxic.prediction).count() / float(val_set.count())
accuracy_severe_toxic

0.9498703543647364

In [ ]:
#accuracy_test
accuracy_severe_toxic_test = test_severe_toxic.filter(test_severe_toxic.severe_toxic == test_severe_toxic.prediction).count() / float(test_set.count())
accuracy_severe_toxic_test

0.9352331606217616

**accuracy на валидации 0.95, accuracy на тесте 0.94. Результаты на тесте и влидации не силно отличаются, модель не переобучилась**

###obscene

In [ ]:
mlor_obscene = LogisticRegression(featuresCol='result_w2v', labelCol='toxic')
mlorModel_obscene = mlor_obscene.fit(train_df_w2v)

In [ ]:
predictions_obscene = mlorModel_obscene.transform(val_df_w2v)

In [ ]:
test_obscene = mlorModel_obscene.transform(test_df_w2v)

In [ ]:
#accuracy_val
accuracy_obscene = predictions_obscene.filter(predictions_obscene.obscene == predictions_obscene.prediction).count() / float(val_set.count())
accuracy_obscene

0.9541918755401901

In [ ]:
#accuracy_test
accuracy_obscene_test = test_obscene.filter(test_obscene.obscene == test_obscene.prediction).count() / float(test_set.count())
accuracy_obscene_test

0.9447322970639033

**accuracy на валидации 0.95, accuracy на тесте 0.94. Результаты на тесте и влидации не силно отличаются, модель не переобучилась**

###threat

In [ ]:
mlor_threat = LogisticRegression(featuresCol='result_w2v', labelCol='toxic')
mlorModel_threat = mlor_threat.fit(train_df_w2v)

In [ ]:
predictions_threat = mlorModel_threat.transform(val_df_w2v)

In [ ]:
test_threat = mlorModel_threat.transform(test_df_w2v)

In [ ]:
#accuracy_val
accuracy_threat = predictions_threat.filter(predictions_threat.threat == predictions_threat.prediction).count() / float(val_set.count())
accuracy_threat

0.9403630077787382

In [ ]:
#accuracy_test
accuracy_threat_test = test_threat.filter(test_toxic.threat == test_threat.prediction).count() / float(test_set.count())
accuracy_threat_test

0.9231433506044905

**accuracy на валидации 0.94, accuracy на тесте 0.92. Результаты на тесте и влидации не силно отличаются, модель не переобучилась**

###insult

In [ ]:
mlor_insult = LogisticRegression(featuresCol='result_w2v', labelCol='toxic')
mlorModel_insult = mlor_insult.fit(train_df_w2v)

In [ ]:
predictions_insult = mlorModel_insult.transform(val_df_w2v)

In [ ]:
test_insult = mlorModel_insult.transform(test_df_w2v)

In [ ]:
#accuracy_val
accuracy_insult = predictions_insult.filter(predictions_insult.insult == predictions_insult.prediction).count() / float(val_set.count())
accuracy_insult

0.9515989628349178

In [ ]:
#accuracy_test
accuracy_toxic_insult = test_insult.filter(test_insult.insult == test_insult.prediction).count() / float(test_set.count())
accuracy_toxic_insult

0.9386873920552677

**accuracy на валидации 0.95, accuracy на тесте 0.94. Результаты на тесте и влидации не силно отличаются, модель не переобучилась**

###identity_hate

In [ ]:
mlor_identity_hate = LogisticRegression(featuresCol='result_w2v', labelCol='toxic')
mlorModel_identity_hate = mlor_identity_hate.fit(train_df_w2v)

In [ ]:
predictions_identity_hate = mlorModel_identity_hate.transform(val_df_w2v)

In [ ]:
test_identity_hate = mlorModel_identity_hate.transform(test_df_w2v)

In [ ]:
#accuracy_val
accuracy_identity_hate = predictions_identity_hate.filter(predictions_identity_hate.identity_hate == predictions_identity_hate.prediction).count() / float(val_set.count())
accuracy_identity_hate

0.9429559204840103

In [ ]:
#accuracy_test
accuracy_identity_hate_test = test_identity_hate.filter(test_identity_hate.toxic == test_identity_hate.prediction).count() / float(test_set.count())
accuracy_identity_hate_test

0.9205526770293609

**accuracy на валидации 0.94, accuracy на тесте 0.92. Результаты на тесте и влидации не силно отличаются, модель не переобучилась**